# Merge the individual JSON files into a single CSV table

In [2]:
import os
import json
import pandas
import functools

In [20]:
files = filter(lambda a: a.endswith(".json"), os.listdir("results/"))

doctor_lists = []

for fnm in files:
    with open(os.path.join("results", fnm), "r") as f:
        ds = json.load(f)
    for d in ds:
        d["locality"] = fnm[:-5].split("-")[0]
    doctor_lists.append(ds)

def extend(L1, L2):
    L1.extend(L2)
    return L1
        
all_doctors = functools.reduce(extend, doctor_lists)

In [21]:
len(all_doctors)

6128

In [22]:
all_doctors[2]

{'accepting_new_patients': 'No',
 'addresses': ['Park Avenue Medical Clinic\n4634 Park Ave\nTerrace, BC\nV8G 1V7'],
 'gender': 'Female',
 'locality': 'Terrace',
 'name': 'Bradley, Victoria Margaret',
 'practice': 'General Family Practice',
 'specialities': []}

In [23]:
set(functools.reduce(lambda a, b: a+b, map(lambda a: a["specialities"], all_doctors)))

{'CCFP',
 'CCFP (EM)',
 'Emergency Medicine+',
 'RCPSC - Anatomical Pathology',
 'RCPSC - Cardiology',
 'RCPSC - General Surgery',
 'RCPSC - Internal Medicine',
 'RCPSC - Medical Biochemistry'}

In [24]:
names = [d["name"] for d in all_doctors]
addr = [d["addresses"][0].replace("\n", "//") for d in all_doctors]
gender = [d["gender"] for d in all_doctors]
practice = [d["practice"] for d in all_doctors]
newpat = [d["accepting_new_patients"] for d in all_doctors]

gfp = ["General Family Practice" in p for p in practice]
ccfp = ["CCFP" in d["specialities"] for d in all_doctors]
ccfp_em = ["CCFP (EM)" in d["specialities"] for d in all_doctors]
emer_med = ["Emergency Medicine+" in d["specialities"] for d in all_doctors]
an_path = ["RCPSC - Anatomical Pathology" in d["specialities"] for d in all_doctors]
int_med = ["RCPSC - Internal Medicine" in d["specialities"] for d in all_doctors]
med_bio = ["RCPSC - Medical Biochemistry" in d["specialities"] for d in all_doctors]
gen_surg = ["RCPSC - General Surgery" in d["specialities"] for d in all_doctors]

locality = [d["locality"] for d in all_doctors]

In [25]:
data = pandas.DataFrame(
    {
        "Name": names,
        "Address": addr,
        "Gender": gender,
        "Accepting_New_Patients": newpat,
        "General_Family_Practice": gfp,
        "CCFP": ccfp,
        "CCFP_EM": ccfp_em,
        "Emergency_Medicine": emer_med,
        "Anatomical_Pathology": an_path,
        "Internal_Medicine": int_med,
        "Medical_Biochemistry": med_bio,
        "General_Surgery": gen_surg,
        "Locality": locality
    })

In [26]:
data[:10]

,Accepting_New_Patients,Address,Anatomical_Pathology,CCFP,CCFP_EM,Emergency_Medicine,Gender,General_Family_Practice,General_Surgery,Internal_Medicine,Locality,Medical_Biochemistry,Name
0,No,"2791 First Ave//Terrace, BC//V8G 0G2",False,False,False,False,Male,True,False,False,Terrace,False,"Appleton, Geoffrey Marsden"
1,No,Park Avenue Medical Clinic//4634 Park Ave//Ter...,False,True,False,False,Female,True,False,False,Terrace,False,"Austin, Diana Marie Yvonne"
2,No,Park Avenue Medical Clinic//4634 Park Ave//Ter...,False,False,False,False,Female,True,False,False,Terrace,False,"Bradley, Victoria Margaret"
3,No,Park Avenue Medical Ctr.//400-4634 Park Ave//T...,False,False,False,False,Female,True,False,False,Terrace,False,"De Bruin, Maria Magdalena"
4,No,"Emergency Departement, Mills Memorial Hospital...",False,False,False,False,Male,True,False,False,Terrace,False,"de Wit, Christiaan Cronje"
5,No,Natasha Theresia DeSousa Pro Corp//3844 Dejong...,False,True,True,False,Female,True,False,False,Terrace,False,"DeSousa, Natasha Theresia"
6,No,"PO Box 653 Stn Main//Terrace, BC//V8G 4B8",False,True,False,False,Female,True,False,False,Terrace,False,"Dufour, Adrienne Danelle"
7,No,"300-4634 Park Ave//Terrace, BC//V8G 1V7",False,True,False,False,Male,True,False,False,Terrace,False,"Elemuo, Raphael Emezie"
8,No,"4720 Haugland Ave//Terrace, BC//V8G 2W7",False,False,False,False,Male,True,False,False,Terrace,False,"Fourie, Willem Jacobus"
9,No,"4839 Twedle Ave//Terrace, BC//V8G 4R4",False,False,False,False,Male,True,False,False,Terrace,False,"Frank, Selby Armstrong"


In [27]:
data.to_csv("CPSBC-data-v0.2.2.csv")

In [28]:
data.describe().T

,count,unique,top,freq
Accepting_New_Patients,6128,2,No,5757
Address,6128,4579,Surrey Memorial Hospital//13750 96 Ave//Surrey...,17
Anatomical_Pathology,6128,2,False,6127
CCFP,6128,2,True,3765
CCFP_EM,6128,2,False,5917
Emergency_Medicine,6128,2,False,6127
Gender,6128,2,Male,3515
General_Family_Practice,6128,1,True,6128
General_Surgery,6128,2,False,6127
Internal_Medicine,6128,2,False,6127


In [29]:
cnt = 0
for i, row in data.iterrows():
    if "//" not in row.Address:
        print(i, row.Name, row.Address)
        cnt += 1
        if cnt == 10:
            break

# Add geographical information

In [30]:
import os
import requests
import picogeojson
import re

results = {}

In [31]:
for irow, row in data.iterrows():
    
    if str(irow) in results:
        continue
    
    if irow%100 == 0:
        print(irow)
        with open("bc-geodata.json", "w") as f:
            json.dump(results, f)

    addr = row.Address.split("//")
    for i, part in enumerate(addr):
        if re.match("[0-9]{2}", part) is not None:
            addr = addr[i:]
            break
    
    resp = requests.get("https://apps.gov.bc.ca/pub/geocoder/addresses.json",
                        params = {
                            "addressString": " ".join(addr),
                            "maxResults": 1,
                            "provinceCode": "BC",
                            "locality": row.Locality
                        })
    
    if resp.status_code == 200:
        for pt in picogeojson.result_fromstring(resp.content.decode("utf-8")).point_features():
            results[str(irow)] = {"lon": pt.geometry.coordinates[0],
                                  "lat": pt.geometry.coordinates[1],
                                  "name": row.Name,
                                  "addr": row.Address}
            break # only one

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100


In [32]:
with open("bc-geodata.json", "w") as f:
    json.dump(results, f)

In [33]:
results["0"]

{'addr': '2791 First Ave//Terrace, BC//V8G 0G2',
 'lat': 54.3612572,
 'lon': -128.5569946,
 'name': 'Appleton, Geoffrey Marsden'}

In [34]:
index = []
lon = []
lat = []
names = []
for irow, row in data.iterrows():
    index.append(irow)
    if str(irow) in results:
        if results[str(irow)]["name"] != row.Name:
            raise KeyError(irow)
        lon.append(results[str(irow)]["lon"])
        lat.append(results[str(irow)]["lat"])
        
geotable = pandas.DataFrame(dict(index=index, Longitude=lon, Latitude=lat))

In [35]:
geotable.to_csv("bc-geodata.csv")

In [36]:
merged = pandas.concat([data, geotable[["Longitude", "Latitude"]]], axis=1)

In [37]:
merged.shape

(6128, 15)

In [38]:
merged

,Accepting_New_Patients,Address,Anatomical_Pathology,CCFP,CCFP_EM,Emergency_Medicine,Gender,General_Family_Practice,General_Surgery,Internal_Medicine,Locality,Medical_Biochemistry,Name,Longitude,Latitude
0,No,"2791 First Ave//Terrace, BC//V8G 0G2",False,False,False,False,Male,True,False,False,Terrace,False,"Appleton, Geoffrey Marsden",-128.556995,54.361257
1,No,Park Avenue Medical Clinic//4634 Park Ave//Ter...,False,True,False,False,Female,True,False,False,Terrace,False,"Austin, Diana Marie Yvonne",-128.591107,54.518921
2,No,Park Avenue Medical Clinic//4634 Park Ave//Ter...,False,False,False,False,Female,True,False,False,Terrace,False,"Bradley, Victoria Margaret",-128.591107,54.518921
3,No,Park Avenue Medical Ctr.//400-4634 Park Ave//T...,False,False,False,False,Female,True,False,False,Terrace,False,"De Bruin, Maria Magdalena",-128.591107,54.518921
4,No,"Emergency Departement, Mills Memorial Hospital...",False,False,False,False,Male,True,False,False,Terrace,False,"de Wit, Christiaan Cronje",-128.596195,54.511444
5,No,Natasha Theresia DeSousa Pro Corp//3844 Dejong...,False,True,True,False,Female,True,False,False,Terrace,False,"DeSousa, Natasha Theresia",-128.602199,54.528628
6,No,"PO Box 653 Stn Main//Terrace, BC//V8G 4B8",False,True,False,False,Female,True,False,False,Terrace,False,"Dufour, Adrienne Danelle",-123.265106,49.348210
7,No,"300-4634 Park Ave//Terrace, BC//V8G 1V7",False,True,False,False,Male,True,False,False,Terrace,False,"Elemuo, Raphael Emezie",-128.591107,54.518921
8,No,"4720 Haugland Ave//Terrace, BC//V8G 2W7",False,False,False,False,Male,True,False,False,Terrace,False,"Fourie, Willem Jacobus",-128.596195,54.511444
9,No,"4839 Twedle Ave//Terrace, BC//V8G 4R4",False,False,False,False,Male,True,False,False,Terrace,False,"Frank, Selby Armstrong",-128.603256,54.530874


In [39]:
geotable.describe()

,Latitude,Longitude,index
count,6128.000000,6128.000000,6128.000000
mean,49.446749,-123.118281,3063.500000
std,1.716502,3.607848,1769.145556
min,44.199612,-138.447644,0.000000
25%,49.105334,-123.333765,1531.750000
50%,49.250431,-123.072512,3063.500000
75%,49.493841,-122.652234,4595.250000
max,58.807873,-114.892402,6127.000000


In [40]:
feature_collection = picogeojson.FeatureCollection(
    [picogeojson.Feature(
        picogeojson.Point([row.Longitude, row.Latitude]),
        {"Name": row.Name,
         "Locality": row.Locality,
         "Address": row.Address}) for _, row in merged.iterrows()])

In [41]:
with open("doctors.geojson", "w") as f:
    f.write(picogeojson.tostring(feature_collection))